# Module 03

## Session 05 Data Preparation & Feature Engineering

# Featiure Selection

In this chapter, we will do:
* data: adult.csv
* target: income (>50k or <=50k)
* preprocess:
    1. missing value: simple imputer with constant
    2. one hot encoding: relationship, race, gender
    3. binary encoding: workclass, martial status, occupation, native country
    4. ordinal encoding: education (already encoded)
    5. no treatment: numerical
    6. out: fnlwgt
* random state=10, data splitting 70:30, feature selection select percentile model: logistic regression (max_iter=1000, solver='liblinear', C=10)

## Library

In [33]:
# data
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile

# data splitting
from sklearn.model_selection import train_test_split

# modeling
from sklearn.linear_model import LogisticRegression

# validation
from sklearn.metrics import accuracy_score

## Data

In [5]:
df_adult = pd.read_csv('./datasets/adult.csv')
df_adult

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## EDA: Missing value

In [6]:
df_adult.replace('?', np.nan, inplace=True)

In [7]:
df_adult.isnull().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

## EDA: Categorical Variable

In [8]:
df_adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [9]:
cat_var = df_adult.columns[df_adult.dtypes == object]
cat_var

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'gender', 'native-country', 'income'],
      dtype='object')

In [10]:
df_adult[cat_var].describe()

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
count,46043,48842,48842,46033,48842,48842,48842,47985,48842
unique,8,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,33906,15784,22379,6172,19716,41762,32650,43832,37155


## Preprocessing Scheme

In [11]:
one_hot_encoding = OneHotEncoder()
one_hot_var = ['relationship', 'race', 'gender']

# binary_encoding = ce.BinaryEncoder() --> if no missing value
# pipline --> to include methods
binary_encoding = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('binary encoder', ce.BinaryEncoder())
    ]
)
binary_var = ['workclass', 'marital-status', 'occupation', 'native-country']

In [12]:
transformer = ColumnTransformer(
    [
        ('one hot', one_hot_encoding, one_hot_var),
        ('binary', binary_encoding, binary_var)
    ],
    remainder='passthrough'
)

## Data Splitting

In [13]:
X = df_adult.drop(columns=['income', 'fnlwgt', 'education'])
y = df_adult['income']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=10
)

## Apply Preprocessing Scheme

In [15]:
X_train_prep = pd.DataFrame(transformer.fit_transform(X_train))
X_test_prep = pd.DataFrame(transformer.transform(X_test))

/Users/deris/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [16]:
one_hot_var_names = transformer.transformers_[0][1].get_feature_names()
binary_var_names = transformer.transformers_[1][1]['binary encoder'].get_feature_names()

feature_names = list(one_hot_var_names) + list(binary_var_names) + ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [17]:
X_train_prep.columns = feature_names
X_test_prep.columns = feature_names

In [18]:
X_train_prep

,x0_Husband,x0_Not-in-family,x0_Other-relative,x0_Own-child,x0_Unmarried,x0_Wife,x1_Amer-Indian-Eskimo,x1_Asian-Pac-Islander,x1_Black,x1_Other,...,3_2,3_3,3_4,3_5,3_6,age,education-num,capital-gain,capital-loss,hours-per-week
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,63.0,6.0,0.0,0.0,40.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,18.0,4.0,0.0,0.0,40.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,22.0,9.0,0.0,0.0,20.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,21.0,9.0,0.0,0.0,40.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,37.0,11.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34184,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,68.0,4.0,0.0,0.0,30.0
34185,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,41.0,13.0,0.0,0.0,40.0
34186,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,61.0,12.0,0.0,0.0,2.0
34187,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,34.0,9.0,0.0,0.0,60.0


## Feature Selection

In [26]:
select = SelectPercentile(percentile=50)
select.fit(X_train_prep, y_train)
X_train_selected = select.transform(X_train_prep)
X_test_selected = select.transform(X_test_prep)


/Users/deris/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 21 26] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/deris/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


## Selected Feature

In [27]:
select.get_support()

array([ True,  True, False,  True,  True,  True, False, False,  True,
       False, False,  True,  True, False,  True,  True,  True, False,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True])

In [28]:
feature_names

['x0_Husband',
 'x0_Not-in-family',
 'x0_Other-relative',
 'x0_Own-child',
 'x0_Unmarried',
 'x0_Wife',
 'x1_Amer-Indian-Eskimo',
 'x1_Asian-Pac-Islander',
 'x1_Black',
 'x1_Other',
 'x1_White',
 'x2_Female',
 'x2_Male',
 '0_0',
 '0_1',
 '0_2',
 '0_3',
 '1_0',
 '1_1',
 '1_2',
 '1_3',
 '2_0',
 '2_1',
 '2_2',
 '2_3',
 '2_4',
 '3_0',
 '3_1',
 '3_2',
 '3_3',
 '3_4',
 '3_5',
 '3_6',
 'age',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [31]:
df_selected = pd.DataFrame(
    {
        'var':feature_names,
        'indicator':select.get_support()
    }
)
df_selected[df_selected['indicator'] == True]

,var,indicator
0,x0_Husband,True
1,x0_Not-in-family,True
3,x0_Own-child,True
4,x0_Unmarried,True
5,x0_Wife,True
8,x1_Black,True
11,x2_Female,True
12,x2_Male,True
14,0_1,True
15,0_2,True


## Model

In [35]:
logreg = LogisticRegression(C=10, max_iter=1000, solver='liblinear')
logreg.fit(X_train_selected, y_train)

LogisticRegression(C=10, max_iter=1000, solver='liblinear')

In [37]:
y_pred = logreg.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.845628881457722

In [41]:
logreg.coef_[0]

array([-2.03943158e-01, -5.58197116e-01, -1.70567771e+00, -8.12340364e-01,
        9.28789907e-01, -3.26793028e-01, -2.28085305e+00, -1.60798589e+00,
        6.69426969e-02, -6.06292308e-01, -7.06445974e-01, -1.26344872e+00,
       -1.74126115e+00, -2.87765987e-02,  2.52818982e-02,  3.41961170e-01,
        3.06480068e-04,  6.74115438e-04,  2.95112866e-02])

In [42]:
df_selected[df_selected['indicator'] == True]['var']

0           x0_Husband
1     x0_Not-in-family
3         x0_Own-child
4         x0_Unmarried
5              x0_Wife
8             x1_Black
11           x2_Female
12             x2_Male
14                 0_1
15                 0_2
16                 0_3
18                 1_1
19                 1_2
20                 1_3
33                 age
34       education-num
35        capital-gain
36        capital-loss
37      hours-per-week
Name: var, dtype: object

In [43]:
coef_table = pd.DataFrame(
    {
        'coef':logreg.coef_[0]
    },
    index=df_selected[df_selected['indicator'] == True]['var'].values
)

In [44]:
coef_table

,coef
x0_Husband,-0.203943
x0_Not-in-family,-0.558197
x0_Own-child,-1.705678
x0_Unmarried,-0.812340
x0_Wife,0.928790
x1_Black,-0.326793
x2_Female,-2.280853
x2_Male,-1.607986
0_1,0.066943
0_2,-0.606292
